In [2]:
from branca.element import Template, MacroElement

In [3]:
from xml.etree import ElementTree as ET
import folium
from shapely.geometry import Point, LineString
import pandas as pd
from sqlalchemy import create_engine
from math import radians, cos, sin, sqrt, atan2
import ast

## Lógica para a construção do código

1.  Ler e tratar (EDA) os dados realizados - Feito
2.  Ler e tratar (EDA) os dados planejados
3.  Aplicar regras de filtros
4.  Colocar 
4.  Criar a lógica para conciliar os dados realizados e planejados
5.  Criar e aplicar a regras de negócio 
6.  Criar a lógica para calcular os desvios
7.  Criar a lógica do relatório
8.  Enviar os dados

### Conexão

In [4]:
dbname = 'poc-db'
user = 'adminpoc'
password = 'adminpoc'
host = 'localhost'  # ou o endereço IP do seu servidor de banco de dados
port = '5440' 

In [5]:
engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{dbname}')

In [6]:
ns = {'kml': 'http://www.opengis.net/kml/2.2'}

In [7]:
sql = """ 
select a.execution_date, a.registration, a.description, a.coordinates
    from hp.geodata a
"""

In [8]:
sql_idle = """ 
select a.id, a.num_linha, a.descricao, b.direcao, b.pontos_geolocalizacao
from
    hp.linhas a,
    hp.trajetos b
where a.id = b.linha_id
"""

In [9]:
df_trip_made = pd.read_sql(sql, engine)
df_trip_made['execution_date'] = pd.to_datetime(df_trip_made['execution_date'])

In [10]:
df_trip_idle = pd.read_sql(sql_idle, engine)

### Paths

In [11]:
planned_trip_path = '../notebooks/data/1.1 LINHAS RMTC [2023-11-29].kml'

### Vars

In [12]:
ns = {'kml': 'http://www.opengis.net/kml/2.2'}

### Funções

In [13]:
def read_and_extract_placemarks(file_path, namespace):
    """ 
    :param: file_path: caminho para o arquivo kml
    :return: lista de dicionários com os nomes e coordenadas dos placemarks

    A função lê um arquivo kml e extrai os placemarks, retornando uma lista de dicionários com os nomes e coordenadas dos placemarks.
    """
    tree = ET.parse(file_path)
    root = tree.getroot()
    placemarks = []
    for placemark in root.findall('.//kml:Placemark', namespace):
        name = placemark.find('kml:name', namespace).text if placemark.find('kml:name', namespace) is not None else "Unnamed"
        coordinates = placemark.find('.//kml:coordinates', namespace).text.strip() if placemark.find('.//kml:coordinates', namespace) is not None else "No coordinates"
        placemarks.append({'name': name, 'coordinates': coordinates})
    return placemarks

In [14]:
def parse_coordinates(coordinates_str):
    """
    param: coordinates_str: string de coordenadas (lat, long)
    return: lista de tuplas de coordenadas (long, lat)

    A função recebe uma string de coordenadas (lat, long) e retorna uma lista de tuplas de coordenadas (long, lat).
    """
    coords = coordinates_str.split()
    return [(float(coord.split(',')[1]), float(coord.split(',')[0])) for coord in coords]

In [15]:
def add_placemarks_to_map(placemarks, map):
    """ 
    param: placemarks: lista de dicionários com as informações dos placemarks
    return: map: objeto do tipo folium.Map

    Quando o arquivo conter somente um ponto, por exemplo uma tupla de coordenadas,ele será plotado como um marcador de viagem realizada. Quando houver
    uma lista de coordenadas, oriundas de um arquivo com trajetória, será plotado uma linha entre os pontos.
    """
    for i, placemark in enumerate(placemarks):
        parsed_coords = parse_coordinates(placemark['coordinates'])
        if len(parsed_coords) == 1:  # Para um único ponto
            folium.Marker(
                location=parsed_coords[0],
                icon=folium.Icon(icon='fa-van-shuttle', prefix='fa',color='blue')
            ).add_to(map)
              
        else:  # Para múltiplos pontos (trajetória)
            if 'VOLTA' in placemark['name']:
                            folium.PolyLine(locations=parsed_coords, color='green', weight=2.5, opacity=1, dash_array='5, 5').add_to(map)
            
            else:  # Para outros casos, usa uma linha contínua
                folium.PolyLine(locations=parsed_coords, color='red', weight=2.5, opacity=1).add_to(map)

In [16]:
def haversine(lon1, lat1, lon2, lat2):
    # Raio da Terra em quilômetros
    R = 6371.0
    
    # Conversão de coordenadas de graus para radianos
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    
    # Diferenças nas coordenadas
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    
    # Fórmula de Haversine
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    
    return distance

In [17]:
def extract_coords(coords_list):
    return [[coord['lat'], coord['lng']] for coord in coords_list]

### EDA

#### Dados realizados

In [18]:
df_trip_made.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287 entries, 0 to 286
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   execution_date  287 non-null    datetime64[ns]
 1   registration    287 non-null    object        
 2   description     287 non-null    object        
 3   coordinates     287 non-null    object        
dtypes: datetime64[ns](1), object(3)
memory usage: 9.1+ KB


In [19]:
#verificar dados nulos
df_trip_made.describe()

,execution_date
count,287
mean,2024-03-17 12:12:32.613240320
min,2024-02-26 00:00:00
25%,2024-03-07 00:00:00
50%,2024-03-18 00:00:00
75%,2024-03-27 00:00:00
max,2024-04-07 00:00:00


In [20]:
df_trip_made.isnull().sum()

execution_date    0
registration      0
description       0
coordinates       0
dtype: int64

In [21]:
#verificar quanntidade de registros únicos
df_trip_made['registration'].value_counts()

registration
NLC8D94    30
NLC7694    30
NLB6B14    29
NKN2285    29
NJY3685    29
NKC3574    29
NLC8F04    29
NKE9854    29
NLB6B04    29
SCQ3A44    24
Name: count, dtype: int64

In [22]:
#verificar quanntidade de registros únicos
df_trip_made['description'].value_counts()

description
20368    30
20357    30
20362    29
20386    29
20360    29
20238    29
20364    29
20224    29
20361    29
20805    24
Name: count, dtype: int64

In [23]:
#verificar quanntidade de registros únicos
df_trip_made['execution_date'].value_counts()

execution_date
2024-03-31    10
2024-03-24    10
2024-04-04    10
2024-03-20    10
2024-03-21    10
2024-03-26    10
2024-04-01    10
2024-03-25    10
2024-03-28    10
2024-03-27    10
2024-04-03    10
2024-04-02    10
2024-03-17    10
2024-02-29    10
2024-03-19    10
2024-03-18    10
2024-03-04    10
2024-03-05    10
2024-02-27    10
2024-03-13    10
2024-03-11    10
2024-03-14    10
2024-03-07    10
2024-02-28    10
2024-03-03     9
2024-04-07     9
2024-03-12     9
2024-03-10     9
2024-03-06     9
2024-02-26     2
Name: count, dtype: int64

### Transformations and rules

#### Dados planejados

In [24]:
placemarks_planned = read_and_extract_placemarks(planned_trip_path, ns)

In [25]:
df_planned_trip = pd.DataFrame(placemarks_planned)

In [26]:
df_planned_trip['Código'] = df_planned_trip['name'].apply(lambda x: x.split('-', 1)[0])

In [27]:
df_planned_trip['identificacao'] = df_planned_trip['name'].apply(lambda x: x.split('-', 1)[1])
df_planned_trip['identificacao'] = df_planned_trip['identificacao'].apply(lambda x: x.split('- ', 1)[0])
df_planned_trip['identificacao'] = df_planned_trip['identificacao'].apply(lambda x: x.split(' ', 1)[0])

In [28]:
df_planned_trip['Código'] = df_planned_trip['name'].apply(lambda x: x.split('-', 1)[0])
df_planned_trip['Descrição e Sentido'] = df_planned_trip['name'].apply(lambda x: x.split(' - ', 1)[1] if ' - ' in x else '')
df_planned_trip['Descrição'] = df_planned_trip['Descrição e Sentido'].apply(lambda x: x.rsplit(' - ', 1)[0] if ' - ' in x else x)
df_planned_trip['Sentido'] = df_planned_trip['Descrição e Sentido'].apply(lambda x: x.rsplit(' - ', 1)[1] if ' - ' in x else '')
df_planned_trip.drop('Descrição e Sentido', axis=1, inplace=True)

In [29]:
de_para = {
    '1': 'A', '2': 'B', '3': 'C', '4': 'D', '5': 'E',
    '6': 'F', '7': 'G', '8': 'H', '9': 'I', '21': 'A',
    '22': 'B', '23': 'C', '24': 'D', '25': 'E', '26': 'F',
    '27': 'G', '28': 'H', '29': 'I', '31': 'A', '32': 'B',
    '33': 'C', '34': 'D', '35': 'E', '36': 'F', '37': 'G',
    '38': 'H', '39': 'I'
}

In [30]:
df_planned_trip['sublinha'] = df_planned_trip['identificacao'].apply(lambda x: de_para[x[:-1]])

#### Dados realizados

In [31]:
df_trip_made_example = df_trip_made[(df_trip_made['execution_date'] == '2024-03-04') & (df_trip_made['description'] == '20386')]


In [32]:
df_trip_made_example['coordinates']

192    [[-49.301427, -16.703299], [-49.30132, -16.703...
Name: coordinates, dtype: object

In [33]:
df_planned_trip_example = df_planned_trip[df_planned_trip['Código'].isin(['400', '401', '176', '160'])]

In [34]:
df_planned_trip_example['coordinates']

314    -49.284614,-16.673452,0 -49.284557997,-16.6734...
315    -49.28328886,-16.662090709,0 -49.283702335,-16...
316    -49.284614,-16.673452,0 -49.284557997,-16.6734...
317    -49.283947481,-16.664584563,0 -49.283864278,-1...
318    -49.284546,-16.673481,0 -49.284504489,-16.6734...
319    -49.235273,-16.671546,0 -49.23620854499999,-16...
320    -49.284612,-16.673732,0 -49.284600331,-16.6737...
321    -49.236824038,-16.671738986,0 -49.235918000000...
Name: coordinates, dtype: object

In [35]:
df_trip_idle['num_linha_numerico'] = df_trip_idle['num_linha'].str.extract(r'^(\d+)')

In [36]:
df_trip_idle

,id,num_linha,descricao,direcao,pontos_geolocalizacao,num_linha_numerico
0,1,002A,PARQUE ATHENEU / T ISIDORIA / PAULO GARCIA,ida,"[{'lat': -16.76353, 'lng': -49.27021}, {'lat':...",002
1,1,002A,PARQUE ATHENEU / T ISIDORIA / PAULO GARCIA,volta,"[{'lat': -16.74154, 'lng': -49.19146}, {'lat':...",002
2,2,002B,PARQUE ATHENEU / T. ISIDORIA,ida,"[{'lat': -16.76353, 'lng': -49.27021}, {'lat':...",002
3,2,002B,PARQUE ATHENEU / T. ISIDORIA,volta,"[{'lat': -16.74154, 'lng': -49.19146}, {'lat':...",002
4,3,002C,PC TRINDADE / T ISIDORIA / PRACA CIVICA,ida,"[{'lat': -16.76353, 'lng': -49.27021}, {'lat':...",002
...,...,...,...,...,...,...
596,319,9901,COPA DA ESTRATEGIA - TURMA I,volta,"[{'lat': -16.705151223923, 'lng': -49.30067673...",9901
597,320,9902,COPA DA ESTRATEGIA - TURMA II,ida,"[{'lat': -16.65976138497988, 'lng': -49.261181...",9902
598,320,9902,COPA DA ESTRATEGIA - TURMA II,volta,"[{'lat': -16.705151223923, 'lng': -49.30067673...",9902
599,321,9903,COPA DA ESTRATEGIA - TURMA III,ida,"[{'lat': -16.65976138497988, 'lng': -49.261181...",9903


In [37]:
df_idle_filtrado = df_trip_idle[df_trip_idle['num_linha_numerico'].isin(['400', '401', '176', '160'])]


In [38]:
df_idle_filtrado

,id,num_linha,descricao,direcao,pontos_geolocalizacao,num_linha_numerico
202,95,160A,T. PRACA A / SANTA HELENA / CAIS CAMPINAS,ida,"[{'lat': -16.705151223923, 'lng': -49.30067673...",160
203,95,160A,T. PRACA A / SANTA HELENA / CAIS CAMPINAS,volta,"[{'lat': -16.673411078397, 'lng': -49.28460794...",160
217,102,176A,T. PRACA A / CAIS CAMPINAS / SANTA HELENA,ida,"[{'lat': -16.705151223923, 'lng': -49.30067673...",176
218,102,176A,T. PRACA A / CAIS CAMPINAS / SANTA HELENA,volta,"[{'lat': -16.673411078397, 'lng': -49.28460794...",176
296,148,400A,CIRCULAR - VIA INDEPENDENCIA,ida,"[{'lat': -16.705151223923, 'lng': -49.30067673...",400
297,148,400A,CIRCULAR - VIA INDEPENDENCIA,volta,"[{'lat': -16.673411078397, 'lng': -49.28460794...",400
298,149,400B,T. PRACA A / T. BIBLIA (VIA FEIRA HIPPIE),ida,"[{'lat': -16.705151223923, 'lng': -49.30067673...",400
299,149,400B,T. PRACA A / T. BIBLIA (VIA FEIRA HIPPIE),volta,"[{'lat': -16.673411078397, 'lng': -49.28460794...",400
300,150,401,CIRCULAR - VIA PCA. WALTER SANTOS,ida,"[{'lat': -16.705151223923, 'lng': -49.30067673...",401
301,150,401,CIRCULAR - VIA PCA. WALTER SANTOS,volta,"[{'lat': -16.673411078397, 'lng': -49.28460794...",401


In [39]:
import folium
import ast  # Import necessário para converter string para lista

# A função parse_coordinates já está definida, então vamos reutilizá-la para df_planned_trip_example
# Para df_trip_made_example, precisamos de uma função para converter a string representando uma lista em uma lista real
def convert_string_to_list(coord_str):
    try:
        # Converte a string que representa uma lista em uma lista Python real
        return ast.literal_eval(coord_str)
    except ValueError:
        # Retorna uma lista vazia se houver erro na conversão
        return []
    
def correct_coordinates(coords_list):
    # Inverte cada par de coordenadas na lista
    return [[lat, lon] for [lon, lat] in coords_list]

# Criando o mapa centrado em um ponto médio
mapa = folium.Map(location=[-16.67, -49.25], zoom_start=12)

# Para df_trip_made_example
for index, row in df_trip_made_example.iterrows():
    coords = convert_string_to_list(row['coordinates'])
    coords = correct_coordinates(coords)
    folium.PolyLine(locations=coords, color="green", weight=2.5, opacity=1, popup=f"Trip {index}").add_to(mapa)

# Para df_planned_trip_example
for index, row in df_planned_trip_example.iterrows():
    coords = parse_coordinates(row['coordinates'])
    folium.PolyLine(locations=coords, color="red", weight=2.5, opacity=1, popup=f"Planned {index}").add_to(mapa)

for _, row in df_idle_filtrado.iterrows():
    # Se os dados já estiverem no formato correto, você pode usar diretamente
    coords_list = row['pontos_geolocalizacao']
    # Extraindo as coordenadas
    coords = extract_coords(coords_list)
    # Adicionando ao mapa
    folium.PolyLine(locations=coords, color="blue", weight=2.5, opacity=1, dash_array='5', popup=row['descricao']).add_to(mapa)

legend_html = '''
<div style="position: fixed; 
     bottom: 40px; left: 40px; width: 180px; height: 110px; 
     border:2px solid grey; z-index:9999; font-size:14px;
     background:white;
     border-radius:5px;
     padding:10px;
     opacity:0.9;">
     <strong>Legenda do Mapa</strong><br>
     <div style="background:green; width:20px; height:3px; display:inline-block; margin-right:5px;"></div>Viagem Realizada<br>
     <div style="background:red; width:20px; height:3px; display:inline-block; margin-right:5px;"></div>Viagem Planejada<br>
     <div style="background:blue; width:20px; height:3px; display:inline-block; margin-right:5px;"></div>Trajeto Ocioso
</div>
'''

# Adicionando a legenda ao mapa
mapa.get_root().html.add_child(folium.Element(legend_html))

mapa

